In [1]:
#imports
import pandas as pd
from datasets import load_dataset, Dataset

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_inputs = pd.read_csv('../preprocessed_data/train_inputs.csv')
train_inputs.head()

,original_Text,Punctuation_Counts_before,Diacritic_Counts,cleaned_text,Stemmed_Text,Lemmatized_Text
0,وكان الرئيس الاوكراني المؤقت الكسندر تورتشينو...,146,0,وكان الرئيس الاوكراني المؤقت الكسندر تورتشينوف...,وكان رئس وكر ؤقت كسندر تورتشينوف امر سحب قوت ا...,كان رئيس الاوكراني المؤقت الكسندر تورتشينوف مر...
1,بحلول عام 2050 ستحتاج مصر الي 21 مليار متر مكع...,74,1,بحلول عام 2050 ستحتاج مصر الي 21 مليار متر مكع...,حلل عام 2050 حاج مصر الي 21 لير متر كعب حصت حل...,حلول عام 2050 احتاج مصر ال 21 مليار متر مكعب ح...
2,وذكرت وكاله الانباء المحليه جي ان اس ان جماع...,38,0,وذكرت وكاله الانباء المحليه جي ان اس ان جماعه ...,ذكر وكل باء حله جي ان اس ان جمع جيش حمد تشدد ا...,ذكر كال الانباء المحليه جي ان اس ان جماع جيش م...
3,ووقع اختياره علي واد عمقه 800 متر محاط بثماني ...,378,15,ووقع اختياره علي واد عمقه 800 متر محاط بثماني ...,وقع خير علي واد عمق 800 متر حاط بثم قمم حده تش...,وقع اختيار علي وادي عمق 800 متر محاط بثماني قم...
4,مسلح حوثي في اب وقال المصدر ان المسلحين الحوث...,26,3,مسلح حوثي اب وقال المصدر ان المسلحين الحوثيين ...,سلح حوث اب وقل صدر ان سلح حوث هجم شطء حرك همي ...,مسلح حوثي اب قال مصدر ان مسلح الحوثيين هاجم نش...


In [3]:
train_labels = pd.read_csv('../preprocessed_data/train_labels.csv')
train_labels.head()

,original_Text,Punctuation_Counts_before,Diacritic_Counts,cleaned_text
0,بدأت القوات الأوكرانيه الانسحاب من شبه جزيره ا...,3,0,بدأت القوات الأوكرانيه الانسحاب من شبه جزيره ا...
1,هل سيتم تغيير العباره الشهيره للمؤرخ اليوناني...,8,0,هل سيتم تغيير العباره الشهيره للمؤرخ اليوناني...
2,قالت الشرطه في القطاع الهندي من إقليم كشمير إن...,3,0,قالت الشرطه في القطاع الهندي من إقليم كشمير إن...
3,في عام 816 تجول راهب يدعي كوكاي في المنحدرات...,9,0,في عام 816 تجول راهب يدعي كوكاي في المنحدرات...
4,أكد مصدر في الحراك التهامي لأبناء محافظه الح...,3,0,أكد مصدر في الحراك التهامي لأبناء محافظه الح...


In [4]:
df_x = train_inputs.copy().rename(columns={'cleaned_text': 'text'})
df_y = train_labels.copy().rename(columns={'cleaned_text': 'summary'})
df = pd.concat([df_x['text'], df_y['summary']], axis=1)
df.head()

,text,summary
0,وكان الرئيس الاوكراني المؤقت الكسندر تورتشينوف...,بدأت القوات الأوكرانيه الانسحاب من شبه جزيره ا...
1,بحلول عام 2050 ستحتاج مصر الي 21 مليار متر مكع...,هل سيتم تغيير العباره الشهيره للمؤرخ اليوناني...
2,وذكرت وكاله الانباء المحليه جي ان اس ان جماعه ...,قالت الشرطه في القطاع الهندي من إقليم كشمير إن...
3,ووقع اختياره علي واد عمقه 800 متر محاط بثماني ...,في عام 816 تجول راهب يدعي كوكاي في المنحدرات...
4,مسلح حوثي اب وقال المصدر ان المسلحين الحوثيين ...,أكد مصدر في الحراك التهامي لأبناء محافظه الح...


In [5]:
train_dataset = Dataset.from_dict(df)

In [6]:
from transformers import AutoTokenizer

model = 'moussaKam/AraBART'
tokenizer = AutoTokenizer.from_pretrained(model)

In [7]:
max_input_length = 1024
max_target_length = 128

# Tokenizes the input dataframe to match the used AraBART model
# Returns a dictionary {input_ids, attention_mask, summary}
def tokenize(df):
    model_input = [row for row in df['text']]
    model_input = tokenizer(model_input, max_length=max_input_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(df['summary'], max_length=max_target_length, truncation=True)
    
    model_input['summary'] = labels['input_ids']
    return model_input


### Different lengths??

In [8]:
tokenized_data = tokenize(train_dataset)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [14]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

pre_trained_model = AutoModelForSeq2SeqLM.from_pretrained(model)

In [10]:
batch_size = 4
model_name = 'AraBART'
arguments = Seq2SeqTrainingArguments(
    model_name,
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    num_train_epochs=3,
    predict_with_generate=True
)

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=pre_trained_model)

In [22]:
trainer = Seq2SeqTrainer(
    pre_trained_model,
    arguments,
    train_dataset=tokenized_data,
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [23]:
# trainer.train()